# Predicting 1-Minute Stock Direction Using Machine Learning

This notebook builds a machine learning model to predict whether
the next one-minute candle will close **higher or lower**
for HDFC Bank stock.

We will:

1. Load minute-level price data
2. Engineer technical features
3. Train Logistic Regression
4. Convert probabilities into trading signals
5. Backtest the strategy with transaction costs
6. Interpret model coefficients

> Goal: Understand whether ML captures directional signal,
not guarantee profitability.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

## Step 1 — Load Data

We load the 1-minute OHLCV data for HDFC Bank.
Each row represents one minute of trading activity.

Important columns:
- open
- high
- low
- close
- volume


In [2]:
df = pd.read_csv("HDFCBANK_minute.csv")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

df = df.sort_values('date').reset_index(drop=True)
df.head()


,date,open,high,low,close,volume
0,2015-02-02 09:15:00,532.90,532.90,530.20,530.90,12719
1,2015-02-02 09:16:00,531.55,531.90,530.75,530.75,9437
2,2015-02-02 09:17:00,530.75,531.70,530.75,531.45,3500
3,2015-02-02 09:18:00,531.45,531.45,530.40,530.50,5203
4,2015-02-02 09:19:00,530.50,530.75,529.80,529.80,3386


## Step 2 — Feature Engineering

We create features commonly used in quantitative trading:

- returns & log-returns
- candle range
- close-open distance
- volume change
- moving averages
- EMA
- volatility
- RSI (14)
- MACD

These help the model estimate directional bias.


In [3]:
df['return']=df['close'].pct_change() # percentage change
# log_return = ln(close_now / close_previous)
df['log_return']=np.log(df['close']/df['close'].shift(1)) #shift 1 means previous rows 

# add range 
df['range']=df['high']-df['low']

#close open 
# if close-open is +ve then bullish green candle
# or if -ve then bearish red candle
df['co']=df['close']-df['open']

#volume change same as return claculation but with volume
df['volume_change']=df['volume'].pct_change()


# adding indicators in our excel 
# 1st is SMA simple moving average
df['sma_20']=df['close'].rolling(window=20).mean()
df['sma_50']=df['close'].rolling(window=50).mean()
# difference etween fast and slow line of ema 
df['sma_20_50_diff']=df['sma_20']-df['sma_50']

# 2nd is EMA indicator
df['ema_12']=df['close'].ewm(span=12,adjust=False).mean()
df['ema_26']=df['close'].ewm(span=26,adjust=False).mean()

df['ema_12_26_diff']=df['ema_12']-df['ema_26']

#3rd indicator is rolling volatility
# Rolling volatility (standard deviation of log returns)
df['vol_20'] = df['log_return'].rolling(window=20).std()
df['vol_50'] = df['log_return'].rolling(window=50).std()

#4th indicator

window = 14

delta = df['close'].diff()

# Gains (positive changes) and losses (negative changes)
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# Average gain and loss
avg_gain = gain.rolling(window=window).mean()
avg_loss = loss.rolling(window=window).mean()

# Relative Strength (avoid division by zero using +1e-9)
rs = avg_gain / (avg_loss + 1e-9)

# RSI formula
df['rsi_14'] = 100 - (100 / (1 + rs))

# MACD line
df['macd'] = df['ema_12'] - df['ema_26']

# Signal line (9-period EMA of MACD)
df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

# Histogram
df['macd_hist'] = df['macd'] - df['macd_signal']


# print(df.columns)





## Step 3 — Define Target

Target = 1  
if next candle close > current close  
else 0

This makes it a **binary classification** problem.


In [4]:
df['target'] = (df['close'].shift(-1) > df['close']).astype(int)

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna().reset_index(drop=True)


## Step 4 — Train / Test Split and Scaling

We use:
- 80% history → training
- 20% → testing (future data)

Features are standardized
so no feature dominates due to scale.


In [5]:
feature_cols = [
    'return','log_return','range','co','volume_change',
    'sma_20','sma_50','sma_20_50_diff',
    'ema_12','ema_26','ema_12_26_diff',
    'vol_20','vol_50',
    'rsi_14',
    'macd','macd_signal','macd_hist'
]

X = df[feature_cols]
y = df['target']

train_size = int(len(df)*0.8)

X_train = X.iloc[:train_size]
X_test  = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test  = y.iloc[train_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


## Step 5 — Train Logistic Regression

Logistic Regression predicts the probability
that the next candle closes higher.


In [6]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

y_pred = log_reg.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.5327012653395989

Confusion Matrix:
 [[79588 21834]
 [68757 23682]]


## Step 6 — Probability Threshold

Instead of predicting up/down directly,
we use the predicted probability.

We only trade when confidence > threshold (e.g., 0.6).
This reduces false trades.


In [7]:
y_prob = log_reg.predict_proba(X_test_scaled)[:, 1]
thresholds=[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8]
for threshold in thresholds:
    print(f" for threshold = {threshold} we have our model details as :")
    y_pred_custom = (y_prob > threshold).astype(int)

    print("Number of trades:", y_pred_custom.sum())
    print("Trade frequency:", y_pred_custom.mean())

    from sklearn.metrics import precision_score, recall_score
    print("UP Precision:", precision_score(y_test, y_pred_custom))
    print("UP Recall:", recall_score(y_test, y_pred_custom))

    hit_rate = (
        (y_pred_custom == 1) & (y_test == 1)
    ).sum() / max((y_pred_custom == 1).sum(), 1)
    print("Hit Rate on Trades:", hit_rate)
    print("\n\n\n")


 for threshold = 0.4 we have our model details as :
Number of trades: 193505
Trade frequency: 0.9981636327059078
UP Precision: 0.47687139867186895
UP Recall: 0.998247492941291
Hit Rate on Trades: 0.47687139867186895




 for threshold = 0.45 we have our model details as :
Number of trades: 160290
Trade frequency: 0.8268295325000903
UP Precision: 0.4833551687566286
UP Recall: 0.8381419097999762
Hit Rate on Trades: 0.4833551687566286




 for threshold = 0.5 we have our model details as :
Number of trades: 45516
Trade frequency: 0.23478678021881658
UP Precision: 0.5203005536514632
UP Recall: 0.2561905689157174
Hit Rate on Trades: 0.5203005536514632




 for threshold = 0.55 we have our model details as :
Number of trades: 8413
Trade frequency: 0.04339707316066666
UP Precision: 0.5348864852014739
UP Recall: 0.04868075163080518
Hit Rate on Trades: 0.5348864852014739




 for threshold = 0.6 we have our model details as :
Number of trades: 2180
Trade frequency: 0.01124517050876659
UP Precis

## Step 7 — Backtesting With Transaction Costs

Trading rule:
- If probability > 0.6 → Buy
- Hold 1 candle
- Exit next close

Transaction cost assumed = 0.02% per trade.


In [8]:
# for backtesting we create a test df 
df_test=df.iloc[train_size:].copy()
df_test=df_test.reset_index(drop=True)

df_test['prob_up']=y_prob
threshold=0.6
df_test['signal']=(df_test['prob_up']>threshold).astype(int)

df_test['future_return'] = (
    df_test['close'].shift(-1) / df_test['close'] - 1
)
df_test=df_test.dropna()
#0.01 for buy and same for sell but 0.01 means 1percent so 0.0002
transaction_cost=0.0002
df_test['strategy_return'] = df_test['signal'] * (df_test['future_return']-transaction_cost)
print("this is cumulative records starting")
print((1+df_test['strategy_return']).cumprod().head(5))
print("this is cumulative records ending")
print((1+df_test['strategy_return']).cumprod().tail(5))
print("this is total percentage returns")
print((1+df_test['strategy_return']).prod()-1)

win_rate = (df_test[df_test['signal'] == 1]['strategy_return'] > 0).mean()
print("Win rate after costs:", win_rate)

avg_trade_return=df_test[df_test['signal']==1]['strategy_return'].mean()
print("Average return per trade:", avg_trade_return)



this is cumulative records starting
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: strategy_return, dtype: float64
this is cumulative records ending
193855    0.83084
193856    0.83084
193857    0.83084
193858    0.83084
193859    0.83084
Name: strategy_return, dtype: float64
this is total percentage returns
-0.1691596098612147
Win rate after costs: 0.46559633027522934
Average return per trade: -8.396096641707033e-05


## Step 8 — Feature Coefficients

Logistic Regression assigns a weight to each feature.
Positive → bullish influence
Negative → bearish influence


In [9]:
# TO LEARN FEATURE COEFFICIENTS 

'''
Create a pandas Series where:
# the values are the model’s learned coefficients
# the labels are the feature names
# Then sort them to see which features influence predictions the most.
# '''
coefs=pd.Series(
    log_reg.coef_[0],
    index=feature_cols
).sort_values()

print(coefs)
#  this will give weights of each feature 


sma_20_50_diff   -0.088146
rsi_14           -0.080217
co               -0.060022
return           -0.028704
log_return       -0.026142
macd_hist        -0.020238
volume_change     0.000168
vol_20            0.006796
vol_50            0.006909
sma_20            0.010185
ema_26            0.010559
ema_12            0.010643
sma_50            0.010683
macd              0.032261
ema_12_26_diff    0.032261
macd_signal       0.040649
range             0.060959
dtype: float64


## Step 9 — Conclusions

- Market directional predictability exists but is weak
- Transaction costs destroy most edges
- Logistic Regression learns short-term mean reversion behavior
- Random Forest may capture nonlinear structure

Next steps:
- Random Forest Classifier
- Hyperparameter tuning
- Feature selection
- Risk management


## Step 10 — Random Forest Classifier
Random Forest is a tree-based ensemble model that can capture
non-linear relationships between technical indicators and price direction.


In [10]:
rf = RandomForestClassifier(
    n_estimators=200,      # number of trees
    max_depth=12,          # limit depth to reduce overfitting
    random_state=42,
    n_jobs=-1              # use all CPU cores
)
 #we do this on scaled set as we can compare it with log reg model here in decison trees it is not necessary to implement it via scaled dataset 
# only we can do thhis with unscaled dataset also
rf.fit(X_train_scaled,y_train)

y_pred_rf=rf.predict(X_test_scaled)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.5328714903977593

Confusion Matrix:
 [[77361 24061]
 [66497 25942]]

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.76      0.63    101422
           1       0.52      0.28      0.36     92439

    accuracy                           0.53    193861
   macro avg       0.53      0.52      0.50    193861
weighted avg       0.53      0.53      0.50    193861



### STEP -11 Random Forest — Probability Thresholds

Instead of predicting up/down directly,
we use the model's probability output.

We will only trade when the probability of UP
is higher than a chosen threshold (e.g., 0.6).


In [11]:
y_prob_rf = rf.predict_proba(X_test_scaled)[:, 1]

thresholds = [0.5,0.55,0.6,0.65,0.7]

for t in thresholds:
    preds = (y_prob_rf > t).astype(int)

    trades = preds.sum()

    print(f"\n===== Threshold = {t} =====")
    print("Number of trades:", trades)
    print("Trade frequency:", trades / len(preds))
    print("UP precision:", precision_score(y_test, preds))
    print("UP recall:", recall_score(y_test, preds))



===== Threshold = 0.5 =====
Number of trades: 50003
Trade frequency: 0.2579322297935118
UP precision: 0.5188088714677119
UP recall: 0.28063912417918846

===== Threshold = 0.55 =====
Number of trades: 7751
Trade frequency: 0.03998225532727057
UP precision: 0.5585085795381242
UP recall: 0.04683088306883458

===== Threshold = 0.6 =====
Number of trades: 475
Trade frequency: 0.0024502091704881332
UP precision: 0.5515789473684211
UP recall: 0.002834301539393546

===== Threshold = 0.65 =====
Number of trades: 8
Trade frequency: 4.126668076611593e-05
UP precision: 0.5
UP recall: 4.327177922738238e-05

===== Threshold = 0.7 =====
Number of trades: 0
Trade frequency: 0.0
UP precision: 0.0
UP recall: 0.0


C:\Users\RHYDHAM SHAH\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### STEP - 12 Random Forest Strategy Backtest


probability(up) above a confidence threshold.
We hold for one bar and pay transaction costs.

1. build a test dataframe
2. generate sugnals based on a threshold
3. compute future return
4. add transaction cost here we take 0.02%
5. compute strategy_return_rf

In [12]:
df_test = df.iloc[train_size:].copy().reset_index(drop=True)

df_test['prob_up_rf'] = y_prob_rf

threshold = 0.6   # you can change later try it for 0.6 also

df_test['signal_rf'] = (df_test['prob_up_rf'] > threshold).astype(int)

# 3 future reurn
df_test['future_return_rf']=df_test['close'].shift(-1)/df_test['close']   - 1 #-1 as gives &return
df_test=df_test.dropna() #last row will be nan

transaction_cost=0.0002

# 5 strategy return
df_test['strategy_return_rf']=df_test['signal_rf']*(df_test['future_return_rf']-transaction_cost)

### STEP-13 Evaluation of backtesting

In [13]:
total_return_rf = (1 + df_test['strategy_return_rf']).prod() - 1
print("Total compounded return:", total_return_rf)

win_rate = (df_test[df_test['signal_rf'] == 1]['strategy_return_rf'] > 0).mean()
print("Win rate:", win_rate)

avg_trade = df_test[df_test['signal_rf']==1]['strategy_return_rf'].mean()
print("Average return per trade:", avg_trade)

print("Number of trades:", df_test['signal_rf'].sum())


Total compounded return: -0.05957011844303106
Win rate: 0.4378947368421053
Average return per trade: -0.0001288713619597088
Number of trades: 475


### feature importance

In [14]:
fi = pd.Series(
    rf.feature_importances_,
    index=feature_cols
).sort_values()

fi


sma_20            0.038568
ema_12            0.038840
ema_26            0.040680
macd_signal       0.043977
ema_12_26_diff    0.044083
sma_50            0.044135
macd              0.044425
sma_20_50_diff    0.046322
volume_change     0.047046
range             0.056831
vol_50            0.059297
vol_20            0.060630
macd_hist         0.061996
rsi_14            0.062344
co                0.091265
return            0.106693
log_return        0.112869
dtype: float64

### I think our data is not suitable for prediction as for 1 minute candles , trends do not hold longer and each candle is not so well followed by a single trend as macd and ema are some of the main things but it doesnt hold on a one minute time frame also here without transaction cost our model do gives positvie results but if added transaction that is a very huges loss as if considered 0.01% for each buy or sell we take 0.02 for buy and sell together so if we have in our test data lets say 200k candles then 200k and lets say our model trade only about 10k times then 10k*200=200% drag just because of transaction cost and our win rate might be 53-54% but that still converts our overall return to negative or negligible 

## Thus lets move towards 5minute not 30 minutes as :
# At 5-Minute : 

Accuracy improves slightly

Win-rate improves

Trades drop

Better Sharpe/edge

Less bleeding from costs

# At 30-Minute

Accuracy improves MORE

Win-rate closer to 55–60%

But very few trades

Results may swing depending on luck

ML may overfit or underfit